### Enter full names of group members:

##### Name A: Jacob Ørmen Clements

In [1]:
import math
import numpy as np
from sympy import prime
from pathlib import Path  # for paths of files
import csv
import copy
import random
from sklearn.metrics.pairwise import cosine_similarity

# ANSI escape codes for colors
class colors:
    red = '\033[91m'
    green = '\033[92m'
    blue = '\033[94m'
    end = '\033[0m'  

### 1. DGIM

#### 1.1. DGIM algorithm

In [2]:
# Default DGIM parameters

stream_path = 'data/my_stream.txt'

# The window size
N = 500

In [3]:
# Here we define helper functions for the DGIM algorithm implmeneted in the cell below
class Bucket:
    def __init__(self, size, timestamp):
        self.size = size
        self.timestamp = timestamp

def merge_buckets(buckets):
    i = 0
    while i < len(buckets) - 1:
        # Count how many buckets of the same size are next to each other
        count = 1
        while (i + count < len(buckets) and
               buckets[i].size == buckets[i + count].size):
            count += 1
        
        # If there are more than two buckets of the same size, merge them
        if count > 2:
            # Merge the two oldest buckets
            merged_size = buckets[i].size + buckets[i + 1].size
            merged_timestamp = buckets[i + 1].timestamp
            # Remove the two oldest buckets
            del buckets[i:i + 2]
            # Insert the merged bucket
            buckets.insert(i, Bucket(merged_size, merged_timestamp))
            # Since we have modified the list, we should not increment i in this case
        else:
            i += count  # Only increment i by the number of buckets of the same size



In [4]:
def dgim_algorithm(stream_bits, N):
    # Create the buckets and initialize the timestamp
    buckets = []
    bucket_list = []  # List to hold lists of timestamps for each bucket size
    current_timestamp = 0
    
    # Loop through the entire data stream, one bit at a time
    with open(stream_path) as f:
        while True:
            bit = f.read(1)
            
            # Clause to break while loop at the end of the stream
            if not bit:
                break
              
            # To-do! update timestamp
            current_timestamp += 1  # Update timestamp for each bit

            # To-do! implement the dgim algorithm here
            if bit == '1':
                # Add timestamp for the new size 1 bucket
                if not bucket_list:
                    bucket_list.append([current_timestamp])
                else:
                    bucket_list[0].append(current_timestamp)
                
                # Add new bucket for this bit
                buckets.append(Bucket(1, current_timestamp))
                
                # Merge buckets if necessary and update timestamps
                merge_buckets(buckets)
                
                # Update timestamps list based on the merged buckets
                # Clear the old timestamps
                bucket_list = [[] for _ in range(len(buckets))]
                for bucket in buckets:
                    # Find the index for the bucket size (2^index = bucket size)
                    index = int(bucket.size).bit_length() - 1
                    # Add the timestamp to the corresponding list
                    if index < len(bucket_list):
                        bucket_list[index].append(bucket.timestamp)
            
            # Remove buckets that are outside the sliding window
            buckets = [bucket for bucket in buckets if current_timestamp - bucket.timestamp < N]
    
    # Filter out empty lists (empty buckets)
    bucket_list = [timestamps for timestamps in bucket_list if timestamps]
    # The end-timestamp is the last timestamp in the size 1 bucket list
    end_timestamp = bucket_list[0][-1] if bucket_list and bucket_list[0] else current_timestamp
    return bucket_list, end_timestamp


In [5]:
bucket = dgim_algorithm(stream_path, N)

In [6]:
print(f"The updated list of timestamps buckets from DGIM algorithm: \n {bucket[0]}")
print(f"The end timestamp: {bucket[1]}")   

The updated list of timestamps buckets from DGIM algorithm: 
 [[1010099], [1010091, 1010096], [1010083, 1010089], [1010063, 1010075], [1010044], [1010006], [1009821, 1009946], [1009688]]
The end timestamp: 1010099


#### 1.2. Query the Bucket 

In [7]:
def actual_count(stream_path, k):
    stream_list = []
    with open(stream_path, 'r') as file:
        for line in file:
            stream_list.extend(list(map(int, line.strip())))

    # Convert the list into a numpy array
    stream_array = np.array(stream_list)
    
    return int(np.sum(stream_array[-k:]))

In [8]:
def dgim_query(bucket, N, k):  
    
    # Extract the buckets and the end timestamp
    bucket_list, end_time_stamp = bucket

    
    # To-do! initialize the different variables
    one_count = 0
    earliest_time_within_k = end_time_stamp - k
    # Keep track of all buckets by their index to know their size when summing up
    # Last bucket in this list will be the one we have to halve
    indexList = []

    # To-do! query the dgim bucket using the k parameters
    # Iterate over the bucket lists, starting with the smallest buckets
    for index, buckets in enumerate(bucket_list):
        # Check each bucket in the current list

        for timestamp in buckets:
            if timestamp > earliest_time_within_k:
                # This bucket is completely within the last k bits
                indexList.append(index)

    one_count += pow(2,indexList.pop())/2
    for elem in indexList:
        one_count += pow(2,elem)

     
    return math.ceil(one_count)



In [9]:
# List of queries
K = [10, 50, 100, 300, 500] 

In [10]:
print("---------------------------------------------------------------")
for k in K:
    dgim_count = dgim_query(bucket, 10000, k)
    true_count = actual_count(stream_path, k)
    
    print(f"The total 1s in the last {k} bits by DGIM: {dgim_count}")
    print(f"The true count of 1s in the last {k} bits: {true_count}")
    print(f"The DGIM error for predicted 1s in the last {k} bits: \
    {round(abs(100*(dgim_count-true_count))/true_count,2)} %")
    print("---------------------------------------------------------------")

---------------------------------------------------------------
The total 1s in the last 10 bits by DGIM: 4
The true count of 1s in the last 10 bits: 5
The DGIM error for predicted 1s in the last 10 bits:     20.0 %
---------------------------------------------------------------
The total 1s in the last 50 bits by DGIM: 25
The true count of 1s in the last 50 bits: 26
The DGIM error for predicted 1s in the last 50 bits:     3.85 %
---------------------------------------------------------------
The total 1s in the last 100 bits by DGIM: 61
The true count of 1s in the last 100 bits: 51
The DGIM error for predicted 1s in the last 100 bits:     19.61 %
---------------------------------------------------------------
The total 1s in the last 300 bits by DGIM: 173
The true count of 1s in the last 300 bits: 150
The DGIM error for predicted 1s in the last 300 bits:     15.33 %
---------------------------------------------------------------
The total 1s in the last 500 bits by DGIM: 269
The true 

### 2. Bloom filters

In [11]:
# Username data for the creation of bloom filters - B
data_file = (Path("data/bloom_username").with_suffix('.csv'))

# Test data to check the functionality and false positive rate
test1_file = (Path("data/test1_username").with_suffix('.csv'))
test2_file = (Path("data/test2_username").with_suffix('.csv'))

# Default bloom filter parameters
bloom_size = 1500000 # parameter N
h = 3 # number of hash functions

In [12]:
# create an array of bloom filter with zeros
B = np.zeros(bloom_size)

In [13]:
B

array([0., 0., 0., ..., 0., 0., 0.])

#### 2.1. Create Bloom filter

In [14]:
def generate_hash(h, N):
    hash_list = []
    
    # To-do! generate a list of hash functions
    # Generate h prime numbers
    primes = set()
    while len(primes) < h:
        primes.add(prime(random.randint(1,1000)))
    primes = list(primes)
    # Generate the hash functions
    for index in range(h):
        p = primes[index]

        # Define the hash function using the provided formula within our outer function
        def hash_func(s, p=p, N=N):  # Default arguments capture the current values of `p` and `N`
            hash_val = 0
            for i, char in enumerate(s):
                hash_val += ord(char)*p**i
            return hash_val %N
    
        hash_list.append(hash_func)

    return hash_list


In [15]:
hashes = generate_hash(h, bloom_size)

In [16]:
def create_bloom_filter(B, hashes, data_file):
    # Provided code gave errors when trying to open the files, hence, I've implemented new code to fix it
    # Open the file using csv.reader
    with open(data_file, mode='r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            # Assuming each row contains a single username as the first element
            name = row[0].strip()
            for hash_func in hashes:
                index = hash_func(name)
                B[index] = 1  # Set the bit at this index to 1
    return B



In [17]:
bloom_array = create_bloom_filter(B, hashes, data_file)

In [18]:
bloom_array

array([1., 1., 1., ..., 0., 0., 1.])

#### 2.2. Verify usernames

In [19]:
def single_verify_username(bloom_array, hashes, new_user):

    # To-do! verify username and return a code of 0 or 1 (1 - username taken and 0 - username available)
    
    # For each hash function, check if the bit at the position it indicates is set to 1
    for hash_func in hashes:
        index = hash_func(new_user)
        if bloom_array[index] == 0:
            return 0  # If any bit is 0, the username has definitely not been taken
    return 1  # If all bits are 1, the username might have been taken (potential false positive)


In [20]:
# Feel free to test different usernames here

new_username = "KazeemTDT4305"

# new_username = "ShambaTDT4305"

In [21]:
user_code = single_verify_username(bloom_array, hashes, new_username)

In [22]:
if user_code == 1:
    print(colors.red + f"Username {new_username} has been taken. Try again!" + colors.end)
elif user_code == 0:
    print(colors.green + f"Username {new_username} is available. Congrats!" + colors.end)
else:
    print(colors.blue + f"Wrong pass code. Please reverify!" + colors.end)  

Username KazeemTDT4305 is available. Congrats!


In [23]:
def group_verify_username(bloom_array, hashes, data_file):
    # Initialize counts
    total_name = 0
    taken_name = 0

    # Open the file using csv.reader for simplicity (same problem as earlier with provided code)
    with open(data_file, mode='r', newline='', encoding='utf-8') as csvfile:
        csvreader = csv.reader(csvfile)
        for row in csvreader:
            # To-do! similar to the single verify, but returns a percentage of usernames taken...
            # ...(In other words seen already by the bloom filter during its creation)
            name = row[0].strip()  # We know each row has one username
            total_name += 1
            # Check if the name is possibly taken
            if all(bloom_array[hash_func(name)] for hash_func in hashes):
                taken_name += 1

    # Calculate the percentage of names taken
    return round((taken_name / total_name) * 100, 2)


In [24]:
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test1_file)
print(f"Percentage of username seen before from test 1: {user_total}%")
print("----------------------------------------------------------")
user_total = group_verify_username(bloom_array, hashes, test2_file)
print(f"Percentage of username seen before from test 2: {user_total}%")
print("----------------------------------------------------------")

----------------------------------------------------------
Percentage of username seen before from test 1: 100.0%
----------------------------------------------------------
Percentage of username seen before from test 2: 23.83%
----------------------------------------------------------


### 3. Flajolet-Martin

In [25]:
def flajolet_martin(input_stream):
    R = 0  # Initialize maximum rightmost zero bit position to 0

    # To-do! Define hash function h(x) = 6x + 1 mod 5
    def h(x):
        return (6*x+1) % 5

    # To-do! Iterate over the input stream and update maximum rightmost zero bit position
    for x in input_stream:
        hash_value = h(x)
        r = 0  # Count the trailing zeros in the binary representation of hash_value
    
        while hash_value > 0:
            if (hash_value & 1) == 0:
                r += 1
                hash_value >>= 1
            else:
                break  # Exit the loop if a 1 is encountered
        
        R = max(R, r)  # Update R to the maximum trailing zeros seen so far

    # Estimate the number of distinct elements
    distinct_estimate = 2 ** R

    return distinct_estimate


In [26]:
# Input stream
input_stream1 = [1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1]
input_stream2 = [1, 3, 2, 1, 2, 3, 4, 3, 1, 2, 3, 1]

# Run the Flajolet-Martin algorithm
distinct_estimate1 = flajolet_martin(input_stream1)
distinct_estimate2 = flajolet_martin(input_stream2)

# Print the estimated number of distinct elements
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 1:", distinct_estimate1)
print("-----------------------------------------------------")
print(f"Distinct elements (estimated) in input stream 2:", distinct_estimate2)
print("-----------------------------------------------------")

-----------------------------------------------------
Distinct elements (estimated) in input stream 1: 2
-----------------------------------------------------
Distinct elements (estimated) in input stream 2: 4
-----------------------------------------------------


### 4. Adword 

#### 4.1. Greedy Algorithm

In [27]:
# User queries
queries = ["big data", "big data", "big data","bloom filters", "bloom filters", "bloom filters",
           "flajolet martin", "flajolet martin", "flajolet martin", "dgim algorithm", "dgim algorithm", "dgim algorithm"]

In [28]:
# Company A B C and D keywords and budget $$$
global_companies = {
        'A': ["big data", "bloom filters", 3],
        'B': ["flajolet martin", 3],
        'C': ["flajolet martin", "dgim algorithm", 3],
        'D': ["big data", 3],
    }

In [29]:
def greedy_algorithm(local_companies, queries):
    # Initialize revenue
    revenue = 0

    # To-do! update revenue using greedy algorithm
    # Go through each query
    for query in queries:
        # Find companies that have the query as a keyword and still have budget
        matching_companies = [company for company, details in local_companies.items()
                              if query in details[:-1] and details[-1] > 0]

        if matching_companies:
            # Select a random company from those that match
            selected_company = random.choice(matching_companies)

            # Deduct 1 from the selected company's budget
            local_companies[selected_company][-1] -= 1

            # Add to the total revenue
            revenue += 1

    return revenue

In [30]:
total_revenue = 0
total_trials = 10
print("Starting trials using Greedy Algorithm...")
print("------------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = greedy_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("------------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Greedy Algorithm...
------------------------------------------------
Trial 1 - Revenue generated: 10
Trial 2 - Revenue generated: 7
Trial 3 - Revenue generated: 10
Trial 4 - Revenue generated: 9
Trial 5 - Revenue generated: 10
Trial 6 - Revenue generated: 8
Trial 7 - Revenue generated: 7
Trial 8 - Revenue generated: 8
Trial 9 - Revenue generated: 11
Trial 10 - Revenue generated: 10
------------------------------------------------
Average revenue generated for all trials:  9.0


#### 4.2. Balance Algorithm

In [31]:
def balance_algorithm(local_companies, queries):
    # Initial revenue
    revenue = 0

    # To-do! update revenue using balance algorithm
    # Go through each query
    for query in queries:
        # Find companies that have the query as a keyword and still have budget left
        matching_companies = {company: details[-1] for company, details in local_companies.items()
                              if query in details[:-1] and details[-1] > 0}

        if matching_companies:
            # Select companies with the largest remaining budget
            max_budget = max(matching_companies.values())
            companies_with_max_budget = [company for company, budget in matching_companies.items()
                                         if budget == max_budget]

            # If there's a tie, select a random company from those with the largest budget
            if companies_with_max_budget:
                selected_company = random.choice(companies_with_max_budget)

                # Deduct 1 from the selected company's budget
                local_companies[selected_company][-1] -= 1

                # Add to the total revenue
                revenue += 1

    return revenue


In [32]:
local_companies = copy.deepcopy(global_companies)
revenue = balance_algorithm(local_companies, queries)

In [33]:
total_revenue = 0
total_trials = 10
print("Starting trials using Balance Algorithm...")
print("-------------------------------------------")
for i in range(total_trials):
    local_companies = copy.deepcopy(global_companies)
    revenue = balance_algorithm(local_companies, queries)
    total_revenue = total_revenue + revenue
    print(f"Trial {i+1} - Revenue generated: {revenue}")
print("-------------------------------------------")   
print("Average revenue generated for all trials: ",total_revenue/total_trials)

Starting trials using Balance Algorithm...
-------------------------------------------
Trial 1 - Revenue generated: 10
Trial 2 - Revenue generated: 9
Trial 3 - Revenue generated: 9
Trial 4 - Revenue generated: 10
Trial 5 - Revenue generated: 9
Trial 6 - Revenue generated: 9
Trial 7 - Revenue generated: 8
Trial 8 - Revenue generated: 10
Trial 9 - Revenue generated: 9
Trial 10 - Revenue generated: 10
-------------------------------------------
Average revenue generated for all trials:  9.3


### 5. Recommender System

In [34]:
# Ratings matrix (each row corresponds to a movie, and each column corresponds to a user)
ratings_matrix = np.array([
    [1, 0, 3, 0, 0, 5, 0, 0, 5, 0, 4, 0],
    [0, 0, 5, 4, 0, 0, 4, 0, 0, 2, 1, 3],
    [2, 4, 0, 1, 2, 0, 3, 0, 4, 3, 5, 0],
    [0, 2, 4, 0, 5, 0, 0, 4, 0, 0, 2, 0],
    [0, 0, 4, 3, 4, 2, 0, 0, 0, 0, 2, 5],
    [1, 0, 3, 0, 3, 0, 0, 2, 0, 0, 4, 0]
])

#### 5.1. User-User Collaborative Filtering

In [35]:
def user_cf(rate_m, tup_mu, neigh):
    # To-do! implement a user-user CF using cosine similarity as distance measure

    # Rewrite the indices to 0-indexing convention
    movie_index, user_index = tup_mu[0]-1,tup_mu[1]-1
    # Transpose the matrix so it follows usual conventions (row = user)
    rateMatrix = rate_m.T
    # Find the similarity matrix (cosine)
    simMatrix = cosine_similarity(rateMatrix)
    # Find indices of N most similar users
    similar_users = np.argsort(-simMatrix[user_index])
    similar_users = similar_users[np.logical_and(similar_users != user_index, simMatrix[user_index, similar_users] < 1)]
    similar_users = similar_users[:neigh]
    # We use fromula given in slides to find the prediction (option 2)
    numerator = 0
    denominator = 0
    for y in similar_users:
        numerator += simMatrix[user_index,y]*rateMatrix[y,movie_index]
        denominator += (simMatrix[user_index,y])
    r = numerator/denominator
    # we round of to the correct amount of decimals in expected output
    prediction = round(r, 2)
    return prediction

In [36]:
# List of tuple of movie rating by users to be predicted e.g (1, 5) refers to the rating of movie 1 by user 5
list_mu_query = [(1, 5), (3, 3)]

# Neighbor selection (|N|)
neigh = 2

In [37]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = user_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (User-User CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 1.42 (User-User CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 1.49 (User-User CF)
-----------------------------------------------------------------


#### 5.2. Item-Item Collaborative Filtering

In [38]:
def item_cf(rate_m, tup_mu, neigh):
    
    # To-do! implement a item-item CF using cosine similarity as distance measure

    # Rewrite the indices to 0-indexing convention
    movie_index, user_index = tup_mu[0]-1,tup_mu[1]-1
    # Find the similarity matrix (cosine)
    simMatrix = cosine_similarity(rate_m)
    # Find indices of N most similar items
    similar_items = np.argsort(-simMatrix[movie_index])
    similar_items = similar_items[np.logical_and(similar_items != movie_index, simMatrix[movie_index, similar_items] < 1)]
    similar_items = similar_items[:neigh]

    # We use fromula given in slides to find the prediction (option 2)
    numerator = 0
    denominator = 0
    for j in similar_items:
        numerator += simMatrix[movie_index,j]*rate_m[j,user_index]
        denominator += (simMatrix[movie_index,j])
    r = numerator/denominator

    # we round of to the correct amount of decimals in expected output
    prediction = round(r, 2)
    return prediction

In [39]:
print("-----------------------------------------------------------------")   
for mu_query in list_mu_query:
    predicted_rating = item_cf(ratings_matrix, mu_query, neigh)
    print(f"The predicted rating of movie {mu_query[0]} by user {mu_query[1]}: {predicted_rating} (Item-Item CF)")
    print("-----------------------------------------------------------------")   

-----------------------------------------------------------------
The predicted rating of movie 1 by user 5: 2.48 (Item-Item CF)
-----------------------------------------------------------------
The predicted rating of movie 3 by user 3: 3.0 (Item-Item CF)
-----------------------------------------------------------------


### Provide concise answers to all 5 cases in the Project 3 description below

#### Case 1

The space complexity of the DGIM algorithm is $O(\log^2 N)$ due to two primary factors. Firstly, the algorithm uses $O(\log N)$ buckets to efficiently manage data within the window size $N$. Secondly, each bucket requires storage for its details: a timestamp indicating the position of the most recent bit and a count of bits within the bucket, both needing $O(\log N)$ bits each. Consequently, with $O(\log N)$ buckets each storing $O(\log N)$ bits for the timestamp and bit count, the total space required becomes as $O(\log N) \times O(\log N) = O(\log^2 N)$. 

#### Case 2

##### Q1:

Yes, there is a possibility that the username "Kazeem" is available. Bloom filters are probabilistic data structures that can tell us if an item is definitely not in a set, but if it says the item is in the set, there is a certain probability that it could be wrong, known as a false positive. To present my case to the site admin, I would explain the concept of false positives in Bloom filters and request a manual check of the username "Kazeem" in their actual database.

##### Q2:

To convince my friend, I will explain that Bloom filters are designed to never have false negatives (meaning if a Bloom filter says a username is available, it definitely is), but they can have false positives. This means that if the social media platform says "KazeemTDT4305" is available, it is.

##### Q3:

This situation could arise if an account has been deactivated but the username has not been released back into the pool for reuse. Some platforms have a grace period before the username of a deleted account can be taken by a new user. To resolve this, you can contact Google support directly through their help center or by selecting the "Contact Us" option on the relevant support page. It would help to provide evidence that the username appears to be available (such as the bounced email indicating no such user exists).


#### Case 3

##### Flajolet-Martin: Increasing Precision

The precision of the Flajolet-Martin algorithm can be enhanced by addressing its variance in estimates. The approach to increasing precision involves:

1. **Repetition and Parallelization**: Run the algorithm multiple times or in parallel using multiple hash functions. By doing so, we generate a set of estimates rather than relying on a single output.

2. **Combining Estimates**: Since direct averaging can be skewed by extreme values, and the median might not be effective because the algorithm's estimates are powers of 2, we need a more robust method to combine the estimates.

3. **Median of Averages Approach**:
   - **Grouping**: First, group the hash functions and calculate the median of the estimates within each group. This step helps to mitigate the impact of outliers within each subset of hash functions.
   - **Averaging Across Groups**: Then, average these medians across the groups to obtain the final estimate. Averaging the medians from each group balances out the variation and leads to a more stable and precise estimate.

Implementing this approach requires careful selection of the number of groups and hash functions per group to optimize the trade-off between precision and computational effort.


#### Case 4

##### Greedy Algorithm:
For this algorithm, the minimum possible revenue is $min_{All possible inputs I} M_{greedy} = 6$. The maximum is $max_{All possible inputs I} M_{greedy} = 12$. Moreover, the optimal number of matchings is $M_{opt} = 12$. From this, we can find the competative ratio for the greedy algorithm. We get Competetive ratio $= \frac{min_{All possible inputs I} M_{greedy}}{M_{opt}} = \frac{6}{12} = \frac{1}{2}$. 


Correspondingly, we have these results for the balance algorithm: $min_{All possible inputs I} M_{balance} = 8$, $max_{All possible inputs I} M_{balance} = 10$. The competative ratio becomes: competative ration $= \frac{min_{All possible inputs I} M_{balance}}{M_{opt}} = \frac{8}{12} = \frac{2}{3}$. 

#### Case 5

By examining the rating patterns in the matrix, I believe that User-User Collaborative Filtering (CF) would probably give a better prediction for movie 1, user 5. This is because I note that, user 5 has similar preferences to user 1, who gave a a low rating to movie 1 (1), and a dissimamilar (quite oppsite) preference so user 11 who gave movie 1 a high rating (4). This suggest that we can see a pattern and that user 5 would give movie 1 a low rating. For Item-Item CF I could not find any real patterns so use, hence, User-User CF is more suitable for this matrix.